In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from tensorflow.keras.layers import GlobalAveragePooling1D, Concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
dp = pd.read_csv("depression_pre.csv")

In [3]:
pre = dp.drop(columns=['Unnamed: 0', 'Post URL', 'Original Content', 'Saved','ID', 'year', 'month'])
pre

,Title,Post Text,Score,Upvote Ratio,Total Comments,Created On
0,"Anyone else feel like their ""fake depressed""?","it's stupid, but i always doubt the fact i hav...",2195,0.99,156,2018-01-28 13:12:11
1,Why don't you talk to us?,My parents keep asking me this question.\nToda...,1652,0.99,136,2018-01-20 17:05:18
2,An old man killed my only way of coping with d...,- be me\n- have chronic depression and social ...,1644,0.98,107,2018-01-26 20:18:05
3,Anyone just want to take a break from life and...,"That's all I want. I wanna go to a park, lay o...",1403,0.99,104,2018-01-17 00:06:20
4,I had to fill out one of those depression test...,And I scored in the 'severe' range. This lead ...,1299,0.99,96,2018-01-08 13:05:09
...,...,...,...,...,...,...
322,Why does depression and anxiety make it so fuc...,Seriously- I can’t tell even if I’ve known the...,1414,1.00,74,2020-01-12 00:23:46
323,Does anyone ever feel physically too weak. Eve...,I’m beginning to wonder if I have a medical co...,1251,1.00,103,2020-02-01 15:29:40
324,It’s not fair.,"I fucking hate it so much. People around me, j...",1222,0.99,121,2020-01-23 13:13:37
325,I’m tired of being tired all the time.,I’m lying in bed exhausted. Tried to do homewo...,1150,1.00,83,2020-01-18 05:11:20


In [4]:
pre_txt = pre[["Post Text"]]

In [5]:
pre_txt['label'] = 'pre'

<ipython-input-5-158ccbbb929c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt['label'] = 'pre'


In [6]:
pre_txt['Post Text'] = pre_txt['Post Text'].values

<ipython-input-6-c49530c1043c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt['Post Text'] = pre_txt['Post Text'].values


In [7]:
pre_txt

,Post Text,label
0,"it's stupid, but i always doubt the fact i hav...",pre
1,My parents keep asking me this question.\nToda...,pre
2,- be me\n- have chronic depression and social ...,pre
3,"That's all I want. I wanna go to a park, lay o...",pre
4,And I scored in the 'severe' range. This lead ...,pre
...,...,...
322,Seriously- I can’t tell even if I’ve known the...,pre
323,I’m beginning to wonder if I have a medical co...,pre
324,"I fucking hate it so much. People around me, j...",pre
325,I’m lying in bed exhausted. Tried to do homewo...,pre


In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128

# pre covid
input_ids = []
attention_masks = []

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
pre_txt['Post Text'].fillna('', inplace=True)
pre_txt.dropna(subset=['Post Text'], inplace=True)

<ipython-input-9-7f9de0dda555>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt['Post Text'].fillna('', inplace=True)
<ipython-input-9-7f9de0dda555>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt.dropna(subset=['Post Text'], inplace=True)


In [10]:
for text in pre_txt['Post Text']:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask= True,
        return_tensors='tf'
    )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids_before = tf.concat(input_ids, axis=0)
attention_masks_before = tf.concat(attention_masks, axis=0)

In [ ]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

bert_model.trainable = False

input_ids = Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
attention_masks = Input(shape=(max_length,), dtype=tf.int32, name="attention_masks")

bert_output = bert_model([input_ids, attention_masks])[0]  # Using index 0 for the pooled output

pooled_output = GlobalAveragePooling1D()(bert_output)


dense = Dense(64, activation='relu')(pooled_output)

output = Dense(2, activation='softmax')(dense)  # Assuming you have 2 classes for classification


model = Model(inputs=[input_ids, attention_masks], outputs=output)


optimizer = Adam(lr=2e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(
    [input_ids_before, attention_masks_before],
    labels_before,
    epochs=5,
    batch_size=32,
    validation_data=([input_ids_before, attention_masks_before], labels_before)
)